In [ ]:
import cv2
import numpy as np

# Load template image
template = cv2.imread('IMG_7875.jpg', 0)
if template is None:
    print("error")
else:
    print("good")
    


# Initialize video capture
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    
    # Get template dimensions
    scale_percent = 10  # resizing to 10% of the original size
    width = int(template.shape[1] * scale_percent / 100)
    height = int(template.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized_template = cv2.resize(template, dim, interpolation = cv2.INTER_AREA)

    h, w = resized_template.shape[:2]
    # just showing template size for debugging
    print("Template size: Height = {}, Width = {}".format(resized_template.shape[0], resized_template.shape[1]))

    # Get the width and height of the video feed
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (frame_width, frame_height))

    # Canny edge detector object thresholds
    canny_threshold1 = 100
    canny_threshold2 = 200

    # Initialize variables for tracking
    prev_top_left = None
    prev_bottom_right = None
    tracking_enabled = False

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

    # Convert frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Canny edge detection
    edges = cv2.Canny(gray_frame, canny_threshold1, canny_threshold2)

    if tracking_enabled:
        # Define search area based on previous detection
        search_area_top_left = (max(0, prev_top_left[0] - w), max(0, prev_top_left[1] - h))
        search_area_bottom_right = (min(frame.shape[1], prev_bottom_right[0] + w), min(frame.shape[0], prev_bottom_right[1] + h))
        search_area = gray_frame[search_area_top_left[1]:search_area_bottom_right[1], search_area_top_left[0]:search_area_bottom_right[0]]

        # checl to see if search area size is compatible with template
        if search_area.shape[0] < h or search_area.shape[1] < w:
            tracking_enabled = False
        else:
            # Apply cross correlation in the search area
            result = cv2.matchTemplate(search_area, resized_template, cv2.TM_SQDIFF_NORMED)

            # Find the minimum correlation value and its location (best match for TM_sqDiff), i found better results when I switched to this
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
            # print("Min correlation value (tracking):", min_val)

            # The best match's top left corner location
            top_left = min_loc

            # Calculate the bottom right corner of the bounding box
            bottom_right = (top_left[0] + w, top_left[1] + h)
           #  print("Rectangle (tracking): Top Left = {}, Bottom Right = {}".format(top_left, bottom_right))

            # Draw the bounding box
            cv2.rectangle(frame, top_left, bottom_right, (255, 0, 0), 2)

            # Update tracking status and previous location
            prev_top_left = top_left
            prev_bottom_right = bottom_right
    else:
        # Apply cross correlation on the entire frame
        result = cv2.matchTemplate(gray_frame, resized_template, cv2.TM_SQDIFF_NORMED)

       
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
       # print("Min correlation value (initial):", min_val)

        # The best match's top left corner location
        top_left = min_loc

        # Calculate the bottom right corner of the bounding box
        bottom_right = (top_left[0] + w, top_left[1] + h)
        # print("Rectangle (initial): Top Left = {}, Bottom Right = {}".format(top_left, bottom_right))

        # Draw the bounding box
        cv2.rectangle(frame, top_left, bottom_right, (255, 0, 0), 2)

    out.write(frame)
    # Display the resulting frame
    cv2.imshow('Face Tracking', frame)

     # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()

out.release()

cv2.destroyAllWindows()


: 